<a href="https://colab.research.google.com/github/ChyYasir/BioSignal_Detection/blob/main/auto_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Configure Git (only once per session)
!git config --global user.email "chyyasir2000@gmail.com"
!git config --global user.name "ChyYasir"


In [ ]:
!git clone https://github.com/ChyYasir/BioSignal_Detection.git


In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Input, Dense, Softmax
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l1
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [ ]:
# @title
import pandas as pd

cesarean_df = pd.read_csv('/content/drive/MyDrive/ML/early_cesarean_features.csv')
induced_cesarean_df = pd.read_csv('/content/drive/MyDrive/ML/early_induced-cesarean_features.csv')
spontaneous_df = pd.read_csv('/content/drive/MyDrive/ML/early_spontaneous_features.csv')

cesarean_df['label'] = 0
induced_cesarean_df['label'] = 1
spontaneous_df['label'] = 2

merged_df = pd.concat([cesarean_df, induced_cesarean_df, spontaneous_df])

merged_df.to_csv('/content/drive/MyDrive/ML/features.csv', index=False)

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/ML/features.csv")

X = data.drop("label", axis=1).values
y = data["label"].values

Parole decision: False. Reason: Prisoner's parole is denied. Please reassess in the future.


In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

num_classes = len(np.unique(y))
y_categorical = to_categorical(y, num_classes=num_classes)

# print(X_scaled)
# print(y_categorical)

In [ ]:
def build_ssae_model(input_dim, num_classes):
  #first autoencoder
  input_layer = Input(shape=(input_dim,))
  encoded1 = Dense(64, activation="relu", activity_regularizer=l1(10e-5))(input_layer)
  decoded1 = Dense(input_dim, activation="sigmoid")(encoded1)

  autoencoder1 = Model(input_layer, decoded1)
  encoder1 = Model(input_layer, encoded1)

  print(autoencoder1.summary())
  print(encoder1.summary())

  #second autoender
  input_encoded1 = Input(shape=(64,))
  encoded2 = Dense(32, activation="relu", activity_regularizer=l1(10e-5))(input_encoded1)
  decoded2 = Dense(64, activation="sigmoid")(encoded2)

  autoencoder2 = Model(input_encoded1, decoded2)
  encoder2 = Model(input_encoded1, encoded2)
  print(autoencoder2.summary())
  print(encoder2.summary())

  #Finall DNN modle with softmax layer for classification
  encoded_layer1 = encoder1(input_layer)
  encoded_layer2 = encoder2(encoded_layer1)
  output_layer = Dense(num_classes, activation="softmax")(encoded_layer2)

  final_model = Model(input_layer, output_layer)

  return final_model, autoencoder1, encoder1, autoencoder2, encoder2





In [ ]:
def train_ssae_model(X_train, encoder1, encoder2):
  # Train the first autoencoder
  autoencoder1 = Model(encoder1.input, encoder1.layers[-1].output)
  autoencoder1.compile(optimizer="adam", loss="mse")
  autoencoder1.fit(X_train, X_train,epochs=50, batch_size=32, shuffle=True, verbose=0)

  X_encoded1 = encoder1.predict(X_train)

  autoencoder2 = Model(encoder2.input, encoder2.layers[-1].output)
  autoencoder2.compile(optimizer="adam", loss="mse")
  autoencoder2.fit(X_encoded1, X_encoded1, epochs=50, batch_size=32, shuffle=True, verbose=0)

In [ ]:
def evaluate_model(X, y, num_folds=10, num_repeats=30):
  accuracy_list = []
  precision_list = []
  recall_list = []
  f1_list = []

  kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

  for repeat in range(num_repeats):
    for train_index, test_index in kf.split(X):
      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index]

      final_model, autoencoder1, encoder1, autoencoder2, encoder2 = build_ssae_model(X_train.shape[1], num_classes)

      train_ssae_model(X_train, encoder1, encoder2)

      final_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
      final_model.fit(X_train, y_train, epochs=50, batch_size=32, shuffle=True, verbose=0)

      y_pred = final_model.predict(X_test)
      y_pred_classes = np.argmax(y_pred, axis=1)
      y_true = np.argmax(y_test, axis=1)

      accuracy_list.append(accuracy_score(y_true, y_pred_classes))
      precision_list.append(precision_score(y_true, y_pred_classes, average="weighted"))
      recall_list.append(recall_score(y_true, y_pred_classes, average="weighted"))
      f1_list.append(f1_score(y_true, y_pred_classes, average="weighted"))

  accuracy = np.mean(accuracy_list)
  precision = np.mean(precision_list)
  recall = np.mean(recall_list)
  f1 = np.mean(f1_list)

  return accuracy, precision, recall, f1



In [ ]:
accuracy, precision, recall, f1 = evaluate_model(X_scaled, y_categorical)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 16)]              0         
                                                                 
 dense (Dense)               (None, 64)                1088      
                                                                 
 dense_1 (Dense)             (None, 16)                1040      
                                                                 
Total params: 2128 (8.31 KB)
Trainable params: 2128 (8.31 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 16)]              0         
                                                                 
 dense 

ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    

# **One Layer**

In [ ]:
def build_ssae_model(input_dim, num_classes):
    # First and only autoencoder
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(64, activation='relu', activity_regularizer=l1(10e-5))(input_layer)
    decoded = Dense(input_dim, activation='sigmoid')(encoded)

    autoencoder = Model(input_layer, decoded)
    encoder = Model(input_layer, encoded)

    print(autoencoder.summary())
    print(encoder.summary())

    # Final DNN model with softmax layer for classification
    encoded_layer = encoder(input_layer)
    output_layer = Dense(num_classes, activation='softmax')(encoded_layer)

    final_model = Model(input_layer, output_layer)

    return final_model, autoencoder, encoder

# Function to train the SSAE model with one autoencoder
def train_ssae_model(X_train, autoencoder):
    # Train the first and only autoencoder with matching input and output dimensions
    autoencoder.compile(optimizer='adam', loss='mse')
    autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, shuffle=True, verbose=0)

# Function to evaluate the model
def evaluate_model(X, y, num_folds=10, num_repeats=30):
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_list = []

    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    for repeat in range(num_repeats):
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            final_model, autoencoder, encoder = build_ssae_model(X_train.shape[1], num_classes)

            # train_ssae_model(X_train, encoder)
            train_ssae_model(X_train, autoencoder)

            final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
            final_model.fit(X_train, y_train, epochs=50, batch_size=32, shuffle=True, verbose=0)

            y_pred = final_model.predict(X_test)
            y_pred_classes = np.argmax(y_pred, axis=1)
            y_true = np.argmax(y_test, axis=1)

            accuracy_list.append(accuracy_score(y_true, y_pred_classes))
            precision_list.append(precision_score(y_true, y_pred_classes, average='weighted'))
            recall_list.append(recall_score(y_true, y_pred_classes, average='weighted'))
            f1_list.append(f1_score(y_true, y_pred_classes, average='weighted'))

    accuracy = np.mean(accuracy_list)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f1 = np.mean(f1_list)

    return accuracy, precision, recall, f1

# Evaluate the model using 10-fold cross-validation with 30 repetitions
accuracy, precision, recall, f1 = evaluate_model(X_scaled, y_categorical)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 16)]              0         
                                                                 
 dense_3 (Dense)             (None, 64)                1088      
                                                                 
 dense_4 (Dense)             (None, 16)                1040      
                                                                 
Total params: 2128 (8.31 KB)
Trainable params: 2128 (8.31 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 16)]              0         
                                                                 
 dens

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 16)]              0         
                                                                 
 dense_6 (Dense)             (None, 64)                1088      
                                                                 
 dense_7 (Dense)             (None, 16)                1040      
                                                                 
Total params: 2128 (8.31 KB)
Trainable params: 2128 (8.31 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 16)]              0         
                                                                 
 dens

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 [==============================] - 0s 60ms/step
Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 16)]              0         
                                                                 
 dense_12 (Dense)            (None, 64)                1088      
                                                                 
 dense_13 (Dense)            (None, 16)                1040      
                                                                 
Total params: 2128 (8.31 KB)
Trainable params: 2128 (8.31 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 16)]              0         
                 

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 [==============================] - 0s 63ms/step
Model: "model_901"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_301 (InputLayer)      [(None, 16)]              0         
                                                                 
 dense_900 (Dense)           (None, 64)                1088      
                                                                 
 dense_901 (Dense)           (None, 16)                1040      
                                                                 
Total params: 2128 (8.31 KB)
Trainable params: 2128 (8.31 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Model: "model_902"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_301 (InputLayer)      [(None, 16)]              0         
               

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 [==============================] - 0s 62ms/step
Accuracy: 0.8522
Precision: 0.7424
Recall: 0.8522
F1-score: 0.7916


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **Two Layer**

In [ ]:


# Function to build SSAE model with two encoders
def build_ssae_model(input_dim, num_classes):
    # First autoencoder
    input_layer = Input(shape=(input_dim,))
    encoded1 = Dense(64, activation='relu', activity_regularizer=l1(10e-5))(input_layer)
    decoded1 = Dense(input_dim, activation='sigmoid')(encoded1)

    autoencoder1 = Model(input_layer, decoded1)
    encoder1 = Model(input_layer, encoded1)

    print(autoencoder1.summary())
    print(encoder1.summary())

    # Second autoencoder
    input_encoded1 = Input(shape=(64,))
    encoded2 = Dense(32, activation='relu', activity_regularizer=l1(10e-5))(input_encoded1)
    decoded2 = Dense(64, activation='sigmoid')(encoded2)

    autoencoder2 = Model(input_encoded1, decoded2)
    encoder2 = Model(input_encoded1, encoded2)

    print(autoencoder2.summary())
    print(encoder2.summary())

    # Final DNN model with softmax layer for classification
    encoded_layer1 = encoder1(input_layer)
    encoded_layer2 = encoder2(encoded_layer1)
    output_layer = Dense(num_classes, activation='softmax')(encoded_layer2)

    final_model = Model(input_layer, output_layer)

    return final_model, autoencoder1, encoder1, autoencoder2, encoder2

# Function to train SSAE model with two encoders
def train_ssae_model(X_train, autoencoder1, encoder1, autoencoder2):
    # Train the first autoencoder
    autoencoder1.compile(optimizer='adam', loss='mse')
    autoencoder1.fit(X_train, X_train, epochs=50, batch_size=32, shuffle=True, verbose=0)

    # Encode the input data using the first encoder
    X_encoded1 = encoder1.predict(X_train)

    # Train the second autoencoder using the encoded data from the first encoder
    autoencoder2.compile(optimizer='adam', loss='mse')
    autoencoder2.fit(X_encoded1, X_encoded1, epochs=50, batch_size=32, shuffle=True, verbose=0)

# Function to evaluate the SSAE model with two encoders
def evaluate_model(X, y, num_folds=10, num_repeats=30):
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_list = []

    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    for repeat in range(num_repeats):
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            final_model, autoencoder1, encoder1, autoencoder2, encoder2 = build_ssae_model(X_train.shape[1], num_classes)

            # Train the SSAE model
            train_ssae_model(X_train, autoencoder1, encoder1, autoencoder2)

            final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
            final_model.fit(X_train, y_train, epochs=50, batch_size=32, shuffle=True, verbose=0)

            y_pred = final_model.predict(X_test)
            y_pred_classes = np.argmax(y_pred, axis=1)
            y_true = np.argmax(y_test, axis=1)

            accuracy_list.append(accuracy_score(y_true, y_pred_classes))
            precision_list.append(precision_score(y_true, y_pred_classes, average='weighted'))
            recall_list.append(recall_score(y_true, y_pred_classes, average='weighted'))
            f1_list.append(f1_score(y_true, y_pred_classes, average='weighted'))

    accuracy = np.mean(accuracy_list)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f1 = np.mean(f1_list)

    return accuracy, precision, recall, f1

# Assuming X_scaled and y_categorical are already defined and preprocessed
num_classes = y_categorical.shape[1]

# Evaluate the model using 10-fold cross-validation with 30 repetitions
accuracy, precision, recall, f1 = evaluate_model(X_scaled, y_categorical)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 16)]              0         
                                                                 
 dense (Dense)               (None, 64)                1088      
                                                                 
 dense_1 (Dense)             (None, 16)                1040      
                                                                 
Total params: 2128 (8.31 KB)
Trainable params: 2128 (8.31 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 16)]              0         
                                                                 
 dense 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
1/1 [==============================] - 0s 90ms/step
Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 16)]              0         
                                                                 
 dense_10 (Dense)            (None, 64)                1088      
                                                                 
 dense_11 (Dense)            (None, 16)                1040      
                                                                 
Total params: 2128 (8.31 KB)
Trainable params: 2128 (8.31 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Total params: 4192 (16.38 KB)
Trainable params: 4192 (16.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 64)]              0         
                                                                 
 dense_12 (Dense)            (None, 32)                2080      
                                                                 
Total params: 2080 (8.12 KB)
Trainable params: 2080 (8.12 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
1/1 [==============================] - 0s 116ms/step
Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 16)]  

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "model_1492"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_598 (InputLayer)      [(None, 64)]              0         
                                                                 
 dense_1492 (Dense)          (None, 32)                2080      
                                                                 
 dense_1493 (Dense)          (None, 64)                2112      
                                                                 
Total params: 4192 (16.38 KB)
Trainable params: 4192 (16.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Model: "model_1493"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_598 (InputLayer)      [(None, 64)]              0         
                                                               

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "model_1497"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_600 (InputLayer)      [(None, 64)]              0         
                                                                 
 dense_1497 (Dense)          (None, 32)                2080      
                                                                 
 dense_1498 (Dense)          (None, 64)                2112      
                                                                 
Total params: 4192 (16.38 KB)
Trainable params: 4192 (16.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Model: "model_1498"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_600 (InputLayer)      [(None, 64)]              0         
                                                               

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
